In [1]:
#Takes about 75 minutes to run on the whole dataset

import json
import pandas as pd
from itertools import chain
import collections
from rapidfuzz import process as pr
import numpy as np
from polyfuzz import PolyFuzz

# Opening JSON file
f = open('/home/ferdinand_t/data/CMS_2010_to_June_2022_ENGLISH.json')
# returns JSON object as
# a dictionary
data = json.load(f)
df = pd.DataFrame.from_dict(data)
#df['cleanLastModifiedDate'] = pd.to_datetime(df['lastModifiedDate'])
#df.set_index('cleanLastModifiedDate', inplace = True)
#df.sort_index().head()



2023-04-06 12:02:36.379833: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-06 12:02:41.240784: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-04-06 12:02:41.241025: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-04-06 12:02:41.241037: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

In [2]:
lst_lst_keywords_clean = df['keywordStrings']

    # Lower case
lst_lst_keywords_clean = [list(map(str.casefold, x)) for x in lst_lst_keywords_clean]

    # Split
lst_lst_keywords_clean = [list(chain(*[kw.split(', ') for kw in lst_kw])) for lst_kw in lst_lst_keywords_clean]
lst_lst_keywords_clean = [list(chain(*[kw.split(' - ') for kw in lst_kw])) for lst_kw in lst_lst_keywords_clean]

    # Replace unicode and double spaces by a space
lst_lst_keywords_clean = [list(map(lambda x: x.replace('\xa0', ' '), lst_kw)) for lst_kw in lst_lst_keywords_clean]
lst_lst_keywords_clean = [list(map(lambda x: x.replace('  ', ' '), lst_kw)) for lst_kw in lst_lst_keywords_clean]

    # Replace unwanted characters
lst_lst_keywords_clean = [list(map(lambda x: ''.join(filter(str.isprintable, x)), lst_kw)) for lst_kw in lst_lst_keywords_clean]
lst_lst_keywords_clean = [list(map(lambda x: x.replace('.', ''), lst_kw)) for lst_kw in lst_lst_keywords_clean]
lst_lst_keywords_clean = [list(map(lambda x: x.replace('" ', ''), lst_kw)) for lst_kw in lst_lst_keywords_clean]
lst_lst_keywords_clean = [list(map(lambda x: x.replace('"', ''), lst_kw)) for lst_kw in lst_lst_keywords_clean]
lst_lst_keywords_clean = [list(map(lambda x: x.replace("'", ''), lst_kw)) for lst_kw in lst_lst_keywords_clean]
lst_lst_keywords_clean = [list(map(lambda x: x.replace('keywords: ', ''), lst_kw)) for lst_kw in lst_lst_keywords_clean]

    # Remove leading and trailing whitespaces
lst_lst_keywords_clean = [list(map(lambda x: x.strip(), lst_kw)) for lst_kw in lst_lst_keywords_clean]

    # Remove sentences (keywords that have more than 6 spaces)
n_spaces = 6
lst_lst_keywords_clean = [[kw for kw in lst_kw if kw.count(' ')<n_spaces] for lst_kw in lst_lst_keywords_clean]

    # Outputs length of unique keywords before and after
print('Cleaning step 1 out of 2 DONE. Number of unique keywords went from', len(set(list(chain(*df['keywordStrings'])))), \
    'to', len(set(list(chain(*lst_lst_keywords_clean)))))

Cleaning step 1 out of 2 DONE. Number of unique keywords went from 156730 to 137404


In [3]:
keywords_flat = list(chain(*lst_lst_keywords_clean)) # Flatten list
keywords_freq = collections.Counter(keywords_flat)

    # extract unique ones and remove the empty entry
unique_keywords = list(set(keywords_flat))
if '' in unique_keywords:
    unique_keywords.remove('')



In [4]:
#0.97 should be a parameter, I have found that the lower it is the higher the run time
from polyfuzz import PolyFuzz
from polyfuzz.models import RapidFuzz

model_df_matches = pd.DataFrame()
rapidfuzz_matcher = RapidFuzz(n_jobs=1, score_cutoff = 0.97)
#for i in range(100) :
model = PolyFuzz(rapidfuzz_matcher).match(unique_keywords)
model_df = model.get_matches()
#model_df_matches.append(model_df)

In [49]:
model_df.shape

(137403, 3)

In [5]:
model_df['Freq_From'] = pd.DataFrame([keywords_freq[word] for word in model_df['From']])

In [6]:
model_df['Freq_To'] = pd.DataFrame([keywords_freq[word] for word in model_df['To']])

In [7]:
model_df_96 = model_df[model_df['Similarity'] > 0.96]

In [10]:
bleep = pd.DataFrame([model_df_96.iloc[i,model_df_96.columns.get_loc('From')] if model_df_96.iloc[i,model_df_96.columns.get_loc('Freq_From')] > model_df_96.iloc[i,model_df_96.columns.get_loc('Freq_To')] else model_df_96.iloc[i,model_df_96.columns.get_loc('To')] for i in range(model_df_96.shape[0])])

In [11]:
model_df_96['replacement'] = bleep.values

/home/ferdinand_t/venv/lib64/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
keywords_flat = list(chain.from_iterable(lst_lst_keywords_clean))

In [19]:
haystack = keywords_flat
needles = model_df_96['From']
st = set(needles)
index_96 = [i for i, e in enumerate(haystack) if e in st]

In [64]:
keywords_flat_replace_2 = [model_df_96.iloc[list(model_df_96['From']).index(keywords_flat[i]),5] if keywords_flat[i] in model_df_96['From'].values else keywords_flat[i] for i in index_96]

In [85]:
for (index, replacement) in zip(index_96, keywords_flat_replace_2):
    keywords_flat[index] = replacement

In [88]:
def gen_list_of_lists(original_list, new_structure):
    assert len(original_list) == sum(new_structure), \
    "The number of elements in the original list and desired structure don't match"
    list_of_lists = [[original_list[i + sum(new_structure[:j])] for i in range(new_structure[j])] \
                    for j in range(len(new_structure))]
    return list_of_lists

lst_lst_keywords_replaced = gen_list_of_lists(keywords_flat, [len(x) for x in lst_lst_keywords_clean])


# Outputs length of unique keywords before and after
print('Cleaning step 2 out of 2 DONE. Number of unique keywords went from', len(set(list(chain(*lst_lst_keywords_clean)))), \
    'to', len(set(list(chain(*lst_lst_keywords_replaced)))))

Cleaning step 2 out of 2 DONE. Number of unique keywords went from 137404 to 135956


In [ ]:
#Needed to copy some functions to save the dataset

In [90]:
def clean_categories(df):
    ''' 
    Cleans the category column of data frame df
    1. Gets rid of the dictionary format
    2. Extracts all the main (primary) categories

    '''

    # Makes a new column to get rid of the dictionary format
    df['cleanFocusCategory'] = df['thematicFocusCategory'].apply(lambda x: x['name'] if x is not None else x)

    # Convert all secondary categories into primary categories
    children_dict = {'Architecture':'Culture', 'Design':'Culture', 'Film':'Culture', 'Arts':'Culture', 
                    'Literature':'Culture', 'Music':'Culture', 'Dance':'Culture', 'Theater':'Culture',
                    'Climate':'Nature and Environment',
                    'Conflicts':'Politics', 'Terrorism':'Politics', 
                    'Corruption':'Law and Justice', 'Crime':'Law and Justice', 'Rule of Law':'Law and Justice',
                        'Press Freedom':'Law and Justice', 
                    'Diversity':'Human Rights', 'Freedom of Speech':'Human Rights', 'Equality':'Human Rights', 
                    'Soccer': 'Sports',
                        'Trade':'Business', 'Globalization':'Business', 'Food Security':'Business'
    }

    secondary_cts = [val for val in children_dict.keys()]

    # Replaces
    df['cleanFocusParentCategory'] = df['cleanFocusCategory'].apply(lambda x: children_dict[x] if x in secondary_cts else x)

    return df

In [91]:
df = clean_categories(df)

In [94]:
df['Date'] = pd.to_datetime(df['lastModifiedDate']).apply(lambda x: x.date)

In [96]:
df_subset_new = df[['id', 'lastModifiedDate', 'Date', 'keywordStrings', 'cleanFocusParentCategory', 'cleanFocusCategory', 'teaser']].copy()
df_subset_new['keywordStringsCleanAfterFuzz'] = lst_lst_keywords_replaced
    # Storing the data in JSON format
filepath = '../../data/interim/clean_keywords_' + 'FULL' + '.json'
df_subset_new.to_json(filepath, orient = 'split', compression = 'infer', index = 'true')